# Control de inicios de sesión

Este consumer se dedicará a enviar un email a los usuarios cuando inician sesión. No mantiene ningún tipo de estado, y podríamos arrancar varias instancias para tener alta disponibilidad o mejorar el rendimiento simplemente duplicando el script.

In [ ]:
from confluent_kafka import Consumer, KafkaError
import json

from ejercicios.pokemons import TOPIC_USERS, GROUP_LOGIN_CHECKER

In [ ]:
c = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': GROUP_LOGIN_CHECKER,
    'auto.offset.reset': 'latest'
})
c.subscribe([TOPIC_USERS])

In [ ]:
def send_mail(user_id, name):
    print('Sending email to user {}:{}...'.format(user_id, name))

In [ ]:
while True:
    msg = c.poll(1.0)

    if msg is None:
        continue

    if msg.error():
        print("Consumer error: {}".format(msg.error()))
        continue

    if msg.topic() == TOPIC_USERS:
        if msg.value() is None:
            print('User {} logging off, nothing to do'.format(msg.key()))
        else:
            user = json.loads(msg.value())
            send_mail(user['id'], user['name'])
    
c.close()